In [5]:
import gzip
import json
import glob
import subprocess
import os
from ftplib import FTP
from datetime import datetime

# Config FTP
FTP_HOST = 'financeiro.sommus.com'
FTP_USER = 'sglog'
FTP_PASS = '3qh6B3HOb5B0'
FTP_DIR = '/SommusGestor'
ARQUIVOS_PARA_BAIXAR = 12  # quantidade de arquivos mais recentes
PASTA_LOCAL = "D:/LogEstruturado SG"
SEQ_URL = "http://localhost:5341"

def baixar_arquivos_ftp():
    ftp = FTP(FTP_HOST)
    ftp.login(FTP_USER, FTP_PASS)
    ftp.cwd(FTP_DIR)

    # Lista de arquivos com data
    arquivos_info = []

    def parse_line(line):
        parts = line.split(maxsplit=8)  # tenta no máximo 9 partes
        if len(parts) < 9:
            print(f"Ignorando linha malformada: {line}")
            return

        mes = parts[5]
        dia = parts[6]
        hora_ou_ano = parts[7]
        nome = parts[8]

        # Montar a data (se for hora, assume o ano atual)
        try:
            if ':' in hora_ou_ano:
                ano = datetime.now().year
                data = datetime.strptime(f"{mes} {dia} {ano} {hora_ou_ano}", "%b %d %Y %H:%M")
            else:
                data = datetime.strptime(f"{mes} {dia} {hora_ou_ano}", "%b %d %Y")
            arquivos_info.append((nome, data))
        except Exception as e:
            print(f"Erro ao converter data para linha: {line}\n{e}")

    ftp.retrlines('LIST', parse_line)

    # Filtrar somente arquivos .json.gz e ordenar por data (mais novos primeiro)
    arquivos_filtrados = sorted(
        [(nome, data) for nome, data in arquivos_info if nome.endswith(".json.gz")],
        key=lambda x: x[1],
        reverse=True
    )

    arquivos_selecionados = arquivos_filtrados[:ARQUIVOS_PARA_BAIXAR]

    for nome, data in arquivos_selecionados:
        local_path = os.path.join(PASTA_LOCAL, nome)
        print(f"Baixando {nome} ({data})...")
        with open(local_path, 'wb') as f:
            ftp.retrbinary(f'RETR {nome}', f.write)

    ftp.quit()

def processar_e_ingerir():
    for gz_file in glob.glob(os.path.join(PASTA_LOCAL, "sommusgestor.*.json.gz")):
        print(f"Descompactando e processando: {gz_file}")
        trunc_filename = gz_file.replace(".json.gz", ".trunc.json")

        with gzip.open(gz_file, 'rt', encoding='utf-8') as f_in, open(trunc_filename, 'w', encoding='utf-8') as f_out:
            for line in f_in:
                try:
                    event = json.loads(line)
                    if '@m' in event and len(event['@m']) > 500:
                        event['@m'] = event['@m'][:500] + "..."
                    if 'compra' in event:
                        del event['compra']
                    f_out.write(json.dumps(event) + '\n')
                except Exception as e:
                    print(f"Erro ao processar linha no {gz_file}: {e}")

        print(f"Ingerindo: {trunc_filename}")
        try:
            result = subprocess.run([
                "seqcli",
                "ingest",
                "-i", trunc_filename,
                "--json",
                "-s", SEQ_URL
            ], capture_output=True, text=True)

            if result.returncode != 0:
                print(f"Erro ao ingerir {trunc_filename}: {result.stderr}")
            else:
                print(f"Ingestão concluída: {trunc_filename}")

        except Exception as e:
            print(f"Erro ao executar seqcli: {e}")

        # Remover arquivos temporários
        os.remove(trunc_filename)
        os.remove(gz_file)  # opcional

# Execução
baixar_arquivos_ftp()
processar_e_ingerir()


Baixando sommusgestor.log2025040911.json.gz (2025-04-09 12:00:00)...
Baixando sommusgestor.log2025040910.json.gz (2025-04-09 11:00:00)...
Baixando sommusgestor.log2025040909_001.json.gz (2025-04-09 10:00:00)...
Baixando sommusgestor.log2025040909.json.gz (2025-04-09 09:55:00)...
Baixando sommusgestor.log2025040908_001.json.gz (2025-04-09 09:00:00)...
Baixando sommusgestor.log2025040908.json.gz (2025-04-09 08:52:00)...
Baixando sommusgestor.log2025040907.json.gz (2025-04-09 08:00:00)...
Baixando sommusgestor.log2025040906.json.gz (2025-04-09 07:01:00)...
Baixando sommusgestor.log2025040903.json.gz (2025-04-09 06:30:00)...
Baixando sommusgestor.log2025040902.json.gz (2025-04-09 03:24:00)...
Baixando sommusgestor.log2025040901.json.gz (2025-04-09 02:01:00)...
Baixando sommusgestor.log2025040900.json.gz (2025-04-09 01:19:00)...
Descompactando e processando: D:/LogEstruturado SG\sommusgestor.log2025040900.json.gz
Ingerindo: D:/LogEstruturado SG\sommusgestor.log2025040900.trunc.json
Ingestão